In [ ]:
# LOCAL TEST
import requests

BASE_URL = "http://127.0.0.1:8000"

print(requests.get(f"{BASE_URL}/api/v1/healthcheck").json())

{'status': 'OK', 'timestamp': '2025-08-13T17:23:01.556254+00:00'}


In [35]:
# Evaluate
payload = {
    "user_message": "What is the capital of France?",
    "chat_history": [
        {"role": "user", "content": "Hi"},
        {"role": "assistant", "content": "The capital of Canada is Ottawa"},
    ],
}
resp = requests.post(f"{BASE_URL}/api/v1/evaluate", json=payload, timeout=60)


In [36]:
import json
json.loads(resp.text)['reasoning']

"The User's Current Question asks for the capital of France, which is a new topic not present in the Chat History or Previously Retrieved Documents. The Chat History only mentions the capital of Canada, and the Previously Retrieved Documents do not contain information about the capital of France."

In [31]:
# APP TEST
from databricks.sdk.core import Config
import requests

config = Config(profile="DEFAULT")
token = config.oauth_token().access_token

session = requests.Session()
session.headers.update({
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json",
})

BASE_URL = "https://fastapi-shm-984752964297111.11.azure.databricksapps.com"

response = session.get(
    f"{BASE_URL}/api/v1/healthcheck", 
    allow_redirects=False)

print(response.text)

{"status":"OK","timestamp":"2025-08-13T17:14:14.895568+00:00"}


In [32]:
# Evaluate
payload = {
    "user_message": "What is the capital of France?",
    "chat_history": [
        {"role": "user", "content": "Hi"},
        {"role": "assistant", "content": "The capital of Canada is Ottawa"},
    ],
}
resp = requests.post(f"{BASE_URL}/api/v1/evaluate", json=payload, timeout=60)


In [33]:
resp

<Response [401]>